In [1]:
import re
import random

def mask_random_letters(word):
    masked_word = re.sub(r'\w', lambda x: '_' if random.choice([True, False]) else x.group(0), word)
    return masked_word

# Example usage:
word = "hello"
masked_word = mask_random_letters(word)
print(masked_word)


_e_l_


In [5]:
import random

def mask_random_letters(word):
    masked_word = ''
    for letter in word:
        if random.choice([True, False]):  # Randomly decide whether to mask the letter
            masked_word += '_'
        else:
            masked_word += letter
    return masked_word

# Example usage:
word = "hello"
masked_word = mask_random_letters(word)
print(masked_word)


h_llo


In [9]:
import re
import random

def mask_random_letters(word):
    masked_word = re.sub(r'[a-zA-Z]', lambda x: '_' if random.choice([True, False]) else x.group(0), word)
    return masked_word

# Example usage:
word = "hello"
masked_word = mask_random_letters(word)
print(masked_word)


h__lo


In [11]:
import re

def replace_with_underscore(word, letter):
    replaced_word = re.sub(letter, '_', word)
    return replaced_word

# Example usage:
word = "hello"
letter_to_replace = "l"
new_word = replace_with_underscore(word, letter_to_replace)
print(new_word)


he__o


In [47]:
common_words = [
    "apple", "banana", "orange", "grape", "strawberry", 
    "pineapple", "watermelon", "melon", "peach", "pear", 
    "apricot", "plum", "kiwi", "blueberry", "raspberry", 
    "blackberry", "cherry", "mango", "lemon", "lime", 
    "coconut", "papaya", "fig", "avocado", "nectarine"
]

In [96]:
import numpy as np
import re
from collections import defaultdict
np.random.uniform()

0.7001870900977425

In [83]:
%%timeit
# import re

def replace_with_underscore(word, letters_to_replace):
    pattern = '|'.join(re.escape(letter) for letter in letters_to_replace)
    replaced_word = re.sub(pattern, '_', word)
    return replaced_word

# Example usage:
words_set = map(set, common_words)
for word, word_set in zip(common_words, words_set):
    letters_to_replace = random.sample(list(word_set), max(1, int(np.random.uniform()*len(word_set))))
    replaced_word = replace_with_underscore(word, letters_to_replace)

825 µs ± 84.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [84]:
%%timeit
# import re

def replace_with_underscore(word, letters_to_replace):
    pattern = re.compile('|'.join(re.escape(letter) for letter in letters_to_replace))
    replaced_word = pattern.sub('_', word)
    return replaced_word

# Example usage:
words_set = map(set, common_words)
res = []
for word, word_set in zip(common_words, words_set):
    letters_to_replace = random.sample(list(word_set), max(1, int(np.random.uniform()*len(word_set))))
    replaced_word = replace_with_underscore(word, letters_to_replace)
    res.append(replaced_word)


726 µs ± 18.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [90]:
# %%timeit

# import re

pattern_func = lambda x :  re.compile('|'.join(re.escape(letter) for letter in x))
# def replace_with_underscore(word, letters_to_replace):
#     pattern = pattern_func(letters_to_replace)
#     replaced_word = pattern.sub('_', word)
#     return replaced_word

# Example usage:
words_set = map(set, common_words)
res = []
for word, word_set in zip(common_words, words_set):
    letters_to_replace = random.sample(list(word_set), max(1, int(np.random.uniform()*len(word_set))))
    pattern = pattern_func(letters_to_replace)
    replaced_word = pattern.sub('_', word)
    res.append(replaced_word)

print(res)

['app__', '_anana', '_r_n__', 'g_ape', '___aw____y', 'pine_pple', 'watermel_n', '_e_o_', 'pea_h', 'pe_r', 'aprico_', 'p_um', '_iwi', '___e_e__y', '_as_____y', '_lack_erry', 'ch_rr_', 'mang_', 'l_m__', '_i__', 'coconu_', '____y_', '__g', '___c_d_', 'nec__rine']


In [119]:
%%timeit

def mask(src, percentage=None):
    return [_get_mask(word, percentage) for word in src]

def _get_mask(word, perc=None):
    """
    Takes in a words and masks it acc. to the Hangman rules.
    """
    if perc is None:
        perc = np.random.uniform()

    counter = {}
    for i, ch in enumerate(word):
        if ch in counter:
            counter[ch].append(i)
        else:
            counter[ch] = [i]

    for key in random.sample(list(counter.keys()), max(1, int(perc * len(counter)))):
        del counter[key]

    word = ['_'] * len(word)
    for ch in counter:
        for pos in counter[ch]:
            word[pos] = ch

    return ''.join(word)

res = mask(common_words)
# print(res)

148 µs ± 8.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# USE THESE FUNCTIONS!!

```python
def get_valid_actions(guessed_letters):
    valid_actions = torch.ones((len(guessed_letters), 28)).to('cuda')
    valid_actions[:,  0] = 0.
    valid_actions[:, -1] = 0.
    
    for i, s in enumerate(guessed_letters):
        for char in s:
            idx = ord(char) - ord('a') + 1
            valid_actions[i, idx] = 0.
    
    return valid_actions


#################################################################################################


import gymnasium as gym
from gymnasium import spaces
import numpy as np
from sklearn.utils import shuffle

class HangmanEnv(gym.Env):
    def __init__(self, dataloader, max_seq_len=32, init_counter=0):
        super(HangmanEnv, self).__init__()

        self.dataset = shuffle(dataloader.dataset)
        self.counter = init_counter
        self.max_seq_len = max_seq_len
        self.action_space = spaces.Discrete(26)  # 26 possible actions (a-z)
        self.observation_space = spaces.Box(low=0, high=27, shape=(self.max_seq_len,), dtype=int)

        self.hidden_word = None
        self.word_length = None
        self._reset_attributes()
    
    def _reset_attributes(self):
        self.guessed_letters = set()
        self.remaining_attempts = 6  # Maximum attempts
        self.current_state = np.zeros(self.max_seq_len, dtype=int)  # Initial state
        self.game_over = False

    def reset(self, *, seed=0, options=None):
        self.hidden_word = self.dataset[self.counter]
        self.word_length = len(self.hidden_word)
        self._reset_attributes()
        
        # Increment reset counter
        self.counter += 1

        current_word = ''.join([char if char in self.guessed_letters else '_' for char in self.hidden_word])
        self.current_state = self.word2state(current_word)
        return self.current_state, {'word': current_word, 'hidden_word': self.hidden_word, 'guessed_letters': self.guessed_letters}

    def generate_random_word(self):
        # Replace this with your logic for generating random words
        word_list = self.dataset
        idx = self.counter % len(word_list)
        self.counter += 1
        return word_list[idx]

    def step(self, action):
        if action in self.guessed_letters:
            print("You have already guessed that letter.")
        else:
            self.guessed_letters.add(action)
            if action in self.hidden_word:
                reward = 0
            else:
                reward = 0
                self.remaining_attempts -= 1

        if set(self.hidden_word) <= self.guessed_letters or self.remaining_attempts == 0:
            reward = 1 if set(self.hidden_word) <= self.guessed_letters else 0
            self.game_over = True

        current_word = ''.join([char if char in self.guessed_letters else '_' for char in self.hidden_word])
        self.current_state = self.word2state(current_word)
        return self.current_state, reward, self.game_over, self.game_over, {'word': current_word, 'hidden_word': self.hidden_word, 'guessed_letters': self.guessed_letters}

    def word2state(self, word):
        state = [27 if char == '_' else ord(char) - ord('a') + 1 for char in word]
        while len(state) < self.max_seq_len:
            state.append(0)
        return state
    
    
#################################################################################################
```

In [124]:
def break_vector_array(vector_array, break_array):
    subarrays = []
    subarray = []
    for i in range(len(vector_array)):
        if break_array[i] == 1:
            if subarray:
                subarrays.append(subarray)
            subarray = []
        subarray.append(vector_array[i])
    if subarray:
        subarrays.append(subarray)
    return subarrays

# Example usage:
vector_array = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
break_array = [1, 0, 1, 0, 0, 1, 0, 1, 0, 0]

result = break_vector_array(vector_array, break_array)
print(result)


[[1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0], [8.0, 9.0, 10.0]]


In [128]:
import torch
import torch.nn.functional as F
from torch.multiprocessing import Pool, cpu_count

def break_vector_array(vector_arrays, break_arrays):
    num_arrays = len(vector_arrays)
    subarrays = [[] for _ in range(num_arrays)]

    def process(index):
        vector_array = vector_arrays[index]
        break_array = break_arrays[index]
        subarray = []
        for i in range(len(vector_array)):
            if break_array[i] == 1:
                if subarray:
                    subarrays[index].append(torch.tensor(subarray))
                subarray = []
            subarray.append(vector_array[i])
        if subarray:
            subarrays[index].append(torch.tensor(subarray))

    for i in range(num_arrays):
        process(i) #, vector_arrays, break_arrays, subarrays)

    return subarrays

# Example usage:
vector_arrays = [
    torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]),
    torch.tensor([11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0])
]
break_arrays = [
    torch.tensor([0, 0, 1, 0, 0, 1, 0, 1, 0, 0]),
    torch.tensor([0, 0, 0, 1, 0, 1, 0, 0, 1, 0])
]

result = break_vector_array(vector_arrays, break_arrays)
print(result)


[[tensor([1., 2.]), tensor([3., 4., 5.]), tensor([6., 7.]), tensor([ 8.,  9., 10.])], [tensor([11., 12., 13.]), tensor([14., 15.]), tensor([16., 17., 18.]), tensor([19., 20.])]]


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.distributions.categorical import Categorical


def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer


class Agent(nn.Module):
    def __init__(self, envs):
        super(Agent, self).__init__()
        
        
        # pretrained model outputs raw logits of `expected` word from supervised learning
        self.pretrainedLLM = Transformer(src_vocab=28, d_model=128, max_seq_len=32, N=12, heads=8, dropout=0.1)
        self.pretrainedLLM.load_state_dict(torch.load('./weights/model_weights_03202024'))
        
        
        # Flatten output logits from transformer and feed into feed-forward NN
        self.critic = nn.Sequential(
            layer_init(nn.Linear(32*28, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 1), std=1.0),
        )
        self.actor = nn.Sequential(
            layer_init(nn.Linear(32*28, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 28), std=0.01),
        )

    def get_value(self, x):
        return self.critic(x)

    def get_action_and_value(self, x, action=None):
        logits = self.actor(x)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(x)
